# Исследование надёжности заёмщиков <a id='intro'></a>

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Файл с данными и общая информация 
<a id='step_1'></a>

Импортируем библиотеку pandas:

In [1]:
import pandas as pd

Прочитаем файл с данными и сохраним его в переменной `df`:

In [2]:
df = pd.read_csv('/datasets/data.csv')

Выведем `20` строк, чтобы посмотреть данные:

In [3]:
display(df.head(20))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Просмотрим информацию о таблице:

In [4]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


В таблице 12 столбцов.

Информация о столбцах (из описания к проекту):
* `'children'` — количество детей в семье;
* `'days_employed'` — общий трудовой стаж в днях;
* `'dob_years'` — возраст клиента в годах;
* `'education'` — уровень образования клиента;
* `'education_id'` — идентификатор уровня образования;
* `'family_status'` — семейное положение;
* `'family_status_id'` — идентификатор семейного положения;
* `'gender'` — пол клиента;
* `'income_type'` — тип занятости;
* `'debt'` — имел ли задолженность по возврату кредитов;
* `'total_income'` — ежемесячный доход;
* `'purpose'` — цель получения кредита;

Рассмотрим каждый столбец на уникальные значения, чтобы можно было сделать выводы, опираясь на все найденные аномалии, а не только те, что в первых `20` строках и в таблице `info`.

In [5]:
print(f"Уникальные значения столбца children: {df['children'].unique()}")

Уникальные значения столбца children: [ 1  0  3  2 -1  4 20  5]


In [6]:
print(f"Уникальные значения столбца days_employed: {df['days_employed'].unique()}")
# Что-то не поняла, почему не все показывает (или хотя бы 50 значений)?

Уникальные значения столбца days_employed: [-8437.67302776 -4024.80375385 -5623.42261023 ... -2113.3468877
 -3112.4817052  -1984.50758853]


In [7]:
print(f"Уникальные значения столбца dob_years: {df['dob_years'].unique()}")

Уникальные значения столбца dob_years: [42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]


In [8]:
print(f"Уникальные значения столбца education: {df['education'].unique()}")

Уникальные значения столбца education: ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']


In [9]:
print(f"Уникальные значения столбца education_id: {df['education_id'].unique()}")

Уникальные значения столбца education_id: [0 1 2 3 4]


In [10]:
print(f"Уникальные значения столбца family_status: {df['family_status'].unique()}")

Уникальные значения столбца family_status: ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']


In [11]:
print(f"Уникальные значения столбца family_status_id: {df['family_status_id'].unique()}")

Уникальные значения столбца family_status_id: [0 1 2 3 4]


In [12]:
print(f"Уникальные значения столбца gender: {df['gender'].unique()}")

Уникальные значения столбца gender: ['F' 'M' 'XNA']


In [13]:
print(f"Уникальные значения столбца income_type: {df['income_type'].unique()}")

Уникальные значения столбца income_type: ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']


In [14]:
print(f"Уникальные значения столбца debt: {df['debt'].unique()}")

Уникальные значения столбца debt: [0 1]


In [15]:
print(f"Уникальные значения столбца total_income: {df['total_income'].unique()}")

Уникальные значения столбца total_income: [253875.6394526  112080.01410244 145885.95229686 ...  89672.56115303
 244093.05050043  82047.41889948]


In [16]:
print(f"Уникальные значения столбца purpose: {df['purpose'].sort_values().unique()}")

Уникальные значения столбца purpose: ['автомобили' 'автомобиль' 'высшее образование'
 'дополнительное образование' 'жилье' 'заняться высшим образованием'
 'заняться образованием' 'на покупку автомобиля'
 'на покупку подержанного автомобиля' 'на покупку своего автомобиля'
 'на проведение свадьбы' 'недвижимость' 'образование' 'операции с жильем'
 'операции с коммерческой недвижимостью' 'операции с недвижимостью'
 'операции со своей недвижимостью' 'покупка жилой недвижимости'
 'покупка жилья' 'покупка жилья для сдачи' 'покупка жилья для семьи'
 'покупка коммерческой недвижимости' 'покупка недвижимости'
 'покупка своего жилья' 'получение высшего образования'
 'получение дополнительного образования' 'получение образования'
 'приобретение автомобиля' 'профильное образование' 'ремонт жилью'
 'свадьба' 'свой автомобиль' 'сделка с автомобилем'
 'сделка с подержанным автомобилем' 'строительство жилой недвижимости'
 'строительство недвижимости' 'строительство собственной недвижимости'
 'сыграть с

In [20]:
# Использовала для вывода большого количества строк таблицы
# Сброс ограничений на количество выводимых рядов
#pd.set_option('display.max_rows', None)
# Сброс ограничений на число столбцов
#pd.set_option('display.max_columns', None)
# Сброс ограничений на количество символов в записи
#pd.set_option('display.max_colwidth', None)

**Вывод**

При первом осмотре данных можно сделать следующие выводы:

1. В столбце `children` есть отрицательное значение, а также значение, увеличенное в 10 раз (или это супер-многодетная семья?).
2. В столбце `days_employed` стаж имеет отрицательные, аномально-большие значения, пропуски и тип float (обычно стаж это целое число, обозначающее количество лет (может быть еще месяц и день). 
3. В столбце `dob_years` есть нулевое значение возраста клиента.
4. В столбце `education` используется и верхний и нижний регистр, нужно привести к единому оформлению (нижнему регистру).
5. В столбце `education_id` аномалий не замечено.
6. В столбце `family_status` у одного значения есть верхний регистр. (и не понятно, почему статус женат/замужем и гражданский брак разделены, ведь юридически это одно и тоже).
7. В столбце `family_status_id` аномалий не замечено.
8. В столбце `gender` есть непонятное значение, нужно понять сколько таких строк и что с ними делать.
9. В столбце `income_type` аномалий не замечено. (кто такой компаньон?)
10. В столбце `debt` аномалий не замечено.
11. В столбце `total_income` есть пропуски, а таже значения имеют тип float, его нужно перевести в int, чтобы было удобнее работать с данными.
12. В столбце `purpose` цель кредита имеет слишком длинное название, необходимо выделить категории для этого столюца, чтобы было удобнее работать с данными. 

## Шаг 2. Предобработка данных
<a id='step_2'></a>

### Обработка пропусков
<a id='step_2_1'></a>

Посмотрим количество пропусков в столбцах `'days_employed'` и `'total_income'`:

In [6]:
print(df.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


In [19]:
df[df['days_employed'].isna()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Эти 2 столбца имеют больше всего проблем (пропуски, тип данных), также в столбце `days_employed` нужно обработать отрицательные и аномально-большие значения (так как в столбце должен быть стаж указан в днях, большие значения, скорее всего, указаны в часах, их нужно поделить на 24 (может и в секундах, но тогда бы они были еще больше).

Эти пропуски могут быть связаны с тем, что человек официально не имеет стажа или не указал данные или данные были некорректно введены (например, отрицательные значения все же указаны в днях).

Так как таких строк много, мы не можем их просто удалить, нужно заполнить ежемесячный доход медианным или средним значением по каждому типу занятости, а стаж в соответствии с возрастом (или среднее значение стажа).

**Начнем с `'days_employed'`:**

Напишем программу, которая отрицательное число сделает положительным и переведет большие значения из часов в дни:

In [21]:
def days_employed_time(row):
    """
    Переводит отрицательные значения строки в положительные
    Большие значения делит на 24 часа
    """
    days_employed = row['days_employed']
    try:
        if days_employed < 0:
            return days_employed*(-1)
        elif days_employed > 0:
            return days_employed/24
    except:
        return days_employed

In [22]:
df['days_employed'] = df.apply(days_employed_time, axis=1)

Проверим, изменились ли отрицательные значения на положительные и уменьшились ли большие значения:

In [23]:
display(df.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [24]:
#display(df.sort_values(by='days_employed',ascending = False).head(30))

Пустые значения в `'days_employed'` заполним после после обработки нулевых значений в столбце `'dob_years'`, чтобы можно было опираться на среднее значение возраста в зависимости от типа занятости.

**Удалим нули в `'dob_years'`:**

Напишем цикл, который в зависимости от типа занятости подставляет медианное значение возраста вместо нулевого значения:
Можно и удалить эти строки или заполнить значением 'unknown', так как нам не важен возраст, но я решила, чтобы весь столбец был одного типа заполнить его медианным значением в зависимости от типа занятости:

In [25]:
# print(list(df['income_type'].unique()))

['сотрудник', 'пенсионер', 'компаньон', 'госслужащий', 'безработный', 'предприниматель', 'студент', 'в декрете']


In [25]:
# Как сделать функцию, если мы обращаемся к нескольким столбцам? И есть ли более короткая запись? 
#(потратила на эти 3 строчки два часа, но они мне так и не нравятся)
for type_i in list(df['income_type'].unique()):
    type_i_med = int(df['dob_years'][(df['income_type'] == type_i) & (df['dob_years'] != 0)].median())
    df.loc[(df['income_type'] == type_i) & (df['dob_years'] == 0), 'dob_years'] = type_i_med

Проверяем, остались ли строки в столбце `'dob_years'` со значением 0:

In [26]:
print(df[df['dob_years'] == 0]['dob_years'].count())

0


**Заменим пустые значения в `'days_employed'` и `'total_income'`:**

Пустые значения заменим медианой, т.к. среднее значение может быть нерепрезентативным 
(у Маши 10 яблок, а у Кати 0, а в среднем 5)

In [27]:
print('Медиана по доходам:', df.groupby('income_type')['total_income'].median())
print('Медиана по количеству дней трудового стажа:',df.groupby('income_type')['days_employed'].median())

Медиана по доходам: income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64
Медиана по количеству дней трудового стажа: income_type
безработный        15267.235531
в декрете           3296.759962
госслужащий         2689.368353
компаньон           1547.382223
пенсионер          15217.221094
предприниматель      520.848083
сотрудник           1574.202821
студент              578.751554
Name: days_employed, dtype: float64


Воспользуемся методом `transform`, который по значению `'income_type'` заменит пустые значения в в столбце `'total_income'` и `'days_employed'`

In [28]:
df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform('median'))

In [29]:
df['days_employed'] = df['days_employed'].fillna(df.groupby('income_type')['days_employed'].transform('median'))

Посмотрим, изменилось ли медианное значение:

In [30]:
print('Медиана по доходам:', df.groupby('income_type')['total_income'].median())
print('Медиана по количеству дней трудового стажа:',df.groupby('income_type')['days_employed'].median())

Медиана по доходам: income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64
Медиана по количеству дней трудового стажа: income_type
безработный        15267.235531
в декрете           3296.759962
госслужащий         2689.368353
компаньон           1547.382223
пенсионер          15217.221094
предприниматель      520.848083
сотрудник           1574.202821
студент              578.751554
Name: days_employed, dtype: float64


Проверим, сколько осталось пустых значений:

In [32]:
print(df.isna().sum())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


**Остались столбцы с аномальными значениями (`'children'`, `'gender'`), а также столбцы со значениями в разном регистре (`'education'`, `'family_status'`)**

Воспользуемся методом replace()

Значения, равные 20 детям заменим на 2 (так как значения в столбце варьируются от 0 до 5), а значение -1 заменим на 1, так как, возможно, это была просто опечатка)

Также неуказанное значение пола заменим на "стандартное" М, так как в данном исследовании пол нам не важен, а вот значения дохода, статус и количество детей важны.

In [33]:
df['children'] = df['children'].replace(-1, 1)
df['children'] = df['children'].replace(20, 2)

df['gender'] = df['gender'].replace('XNA', 'M')

Проверим уникальные значения в столбцах, в которых мы поменяли данные:

In [34]:
print(f"Уникальные значения столбца children: {df['children'].unique()}")

Уникальные значения столбца children: [1 0 3 2 4 5]


In [35]:
print(f"Уникальные значения столбца gender: {df['gender'].unique()}")

Уникальные значения столбца gender: ['F' 'M']


Приведем к нижнему регистру с помощью метода lower()

In [36]:
df['education']= df['education'].str.lower()
df['family_status']= df['family_status'].str.lower()

Проверим уникальные значения в столбцах, в которых мы поменяли данные:

In [37]:
print(f"Уникальные значения столбца education: {df['education'].unique()}")

Уникальные значения столбца education: ['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


In [38]:
print(f"Уникальные значения столбца family_status: {df['family_status'].unique()}")

Уникальные значения столбца family_status: ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'не женат / не замужем']


**Вывод**

* Были исправлены аномальные значения в столбце `'days_employed'`, заполнены пропуски в столбцах `'total_income'` и `'dob_years'`

* Замены аномалии и пропуски в столбцах `'children'` и `'gender'`

* А также привели значения в столбцах `'education'` и `'family_status'` к нижнему регистру 

### Замена типа данных
<a id='step_2_2'></a>

Необходимо заменить тип данных в столбцах `'days_employed'` и `'total_income'` с float на int, так как нам не нужны настолько точные значения (да и количество дней обычно целое число, скорее всего ошибка при переводе из двоичной сс или в системе хранятся данные с учетом часов и минут, а доход был указан после вычета налогов) и не нужно лишнее использование памяти для хранения остатка после запятой.

Для этого воспользуемся методом astype():

In [39]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Данные изменены на тип int, теперь с ними удобнее работать.

### Обработка дубликатов
<a id='step_2_3'></a>

Посмотрим, сколько явных дубликатов находится в таблице:

In [41]:
print(df.duplicated().sum())

71


Удалим дубликаты методом drop_duplicates(), а также обновим индексы:

In [42]:
df = df.drop_duplicates().reset_index(drop=True)

Проверим, удалены ли все дублиаты:

In [43]:
print(df.duplicated().sum())

0


**Вывод**

Явные дубликаты удалены, возможно иноформация о заемщиках была занесена дважды, либо было задвоение при соединении таблиц (если их изначально было несколько).

Теперь данные готовы для анализа и ответов на поставленные вопросы.

### Лемматизация
<a id='step_2_4'></a>

Подключим библиотеку с функцией лемматизации:

In [44]:
from pymystem3 import Mystem
m = Mystem()

Просмотрим уникальные значения столбца `'purpose'`, сделаем список, а потом выделим леммы с помощью метода lemmatize():

In [45]:
print(f"Уникальные значения столбца purpose: {df['purpose'].sort_values().unique()}")

Уникальные значения столбца purpose: ['автомобили' 'автомобиль' 'высшее образование'
 'дополнительное образование' 'жилье' 'заняться высшим образованием'
 'заняться образованием' 'на покупку автомобиля'
 'на покупку подержанного автомобиля' 'на покупку своего автомобиля'
 'на проведение свадьбы' 'недвижимость' 'образование' 'операции с жильем'
 'операции с коммерческой недвижимостью' 'операции с недвижимостью'
 'операции со своей недвижимостью' 'покупка жилой недвижимости'
 'покупка жилья' 'покупка жилья для сдачи' 'покупка жилья для семьи'
 'покупка коммерческой недвижимости' 'покупка недвижимости'
 'покупка своего жилья' 'получение высшего образования'
 'получение дополнительного образования' 'получение образования'
 'приобретение автомобиля' 'профильное образование' 'ремонт жилью'
 'свадьба' 'свой автомобиль' 'сделка с автомобилем'
 'сделка с подержанным автомобилем' 'строительство жилой недвижимости'
 'строительство недвижимости' 'строительство собственной недвижимости'
 'сыграть с

In [46]:
purpose = list(df['purpose'].sort_values().unique())
print(purpose)

['автомобили', 'автомобиль', 'высшее образование', 'дополнительное образование', 'жилье', 'заняться высшим образованием', 'заняться образованием', 'на покупку автомобиля', 'на покупку подержанного автомобиля', 'на покупку своего автомобиля', 'на проведение свадьбы', 'недвижимость', 'образование', 'операции с жильем', 'операции с коммерческой недвижимостью', 'операции с недвижимостью', 'операции со своей недвижимостью', 'покупка жилой недвижимости', 'покупка жилья', 'покупка жилья для сдачи', 'покупка жилья для семьи', 'покупка коммерческой недвижимости', 'покупка недвижимости', 'покупка своего жилья', 'получение высшего образования', 'получение дополнительного образования', 'получение образования', 'приобретение автомобиля', 'профильное образование', 'ремонт жилью', 'свадьба', 'свой автомобиль', 'сделка с автомобилем', 'сделка с подержанным автомобилем', 'строительство жилой недвижимости', 'строительство недвижимости', 'строительство собственной недвижимости', 'сыграть свадьбу']


In [47]:
text = ','.join(purpose)
lemmas = m.lemmatize(text)
print(lemmas)

['автомобиль', ',', 'автомобиль', ',', 'высокий', ' ', 'образование', ',', 'дополнительный', ' ', 'образование', ',', 'жилье', ',', 'заниматься', ' ', 'высокий', ' ', 'образование', ',', 'заниматься', ' ', 'образование', ',', 'на', ' ', 'покупка', ' ', 'автомобиль', ',', 'на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', ',', 'на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', ',', 'на', ' ', 'проведение', ' ', 'свадьба', ',', 'недвижимость', ',', 'образование', ',', 'операция', ' ', 'с', ' ', 'жилье', ',', 'операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', ',', 'операция', ' ', 'с', ' ', 'недвижимость', ',', 'операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', ',', 'покупка', ' ', 'жилой', ' ', 'недвижимость', ',', 'покупка', ' ', 'жилье', ',', 'покупка', ' ', 'жилье', ' ', 'для', ' ', 'сдача', ',', 'покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', ',', 'покупка', ' ', 'коммерческий', ' ', 'недвижимость', ',', 'покупка', ' ', 'недвижимость', ',', 'покупка',

Вызовем специальный контейнер Counter из модуля collections для подсчёта числа упоминаний слов в тексте:

In [48]:
from collections import Counter
print(Counter(lemmas))

Counter({' ': 59, ',': 37, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'на': 4, 'свой': 4, 'операция': 4, 'высокий': 3, 'свадьба': 3, 'получение': 3, 'строительство': 3, 'дополнительный': 2, 'заниматься': 2, 'подержать': 2, 'коммерческий': 2, 'жилой': 2, 'для': 2, 'сделка': 2, 'проведение': 1, 'со': 1, 'сдача': 1, 'семья': 1, 'приобретение': 1, 'профильный': 1, 'ремонт': 1, 'собственный': 1, 'сыграть': 1, '\n': 1})


**Вывод**

Проанализировав леммы (самые популярные - это покупка, недвижимость, автомобиль и образование), выделим несколько категорий по целям кредита:

1. 'недвижимость' - Можно выделить также подгруппы по жилой/коммерческой недвижимости, но не везде указан тип, также обычно ипотеки это целевой кредит на жилье, а на коммерческую недвижимость можно оформить и потребительский кредит

2. 'автомобиль' - Также можно выделить подкатегории на покупку поддержанного или нового автомобиля, но это также не важно, т.к. на поддержанный автомобиль можно взять потребительский кредит, а целевой обычно берут но новый.

3. 'образование' - Есть уточнения, высшее ли это образование или дополнительное, но это все можно объединить одним словом - 'образование'.

4. 'свадьба' - Кредит на проведение свадьбы, никто не указывает, какая это свадьба(много ли кредитов бурет на вторые свадьбы, интересно?), да и это не важно, указывают только, что это на проведение свадьбы.

### Категоризация данных
<a id='step_2_5'></a>

Для ответов на вопросы, необходимо подготовить категории по цели кредита, по уровню дохода и по типу семьи (количество детей).

Для каждого из этих столбцов создадим еще один столбец категории, выделив группы (по кол-ву детей, по уровню дохода, по типу цели).

Для удобного анализа все категории будут объектами, а не числами .

После лемматизации можно выделить 4 группы, для этого создадим функцию, которой будем передавать строку, найдем в ней лемму и добавим в новый столбец группу, в зависимости от леммы:

In [49]:
def purpose_category(row):
    """
    Объединяет в 4 категории по главному слову в цели кредита
    """
    purpose = row['purpose'] 
    
    if 'жиль' in purpose:
        return 'недвижимость'
    
    elif 'недвижимост' in purpose:
        return 'недвижимость'
    
    elif 'автомоб' in purpose:
        return 'автомобиль'
    
    elif 'образован' in purpose:
        return 'образование'
    
    elif 'свадьб' in purpose:
        return 'свадьба'

In [50]:
df['purpose_category'] = df.apply(purpose_category, axis=1)

Проверим, что все строки получили свою категорию и уникальные значения нового столбца:

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 13 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
purpose_category    21454 non-null object
dtypes: int64(7), object(6)
memory usage: 2.1+ MB


In [52]:
print(f"Уникальные значения столбца purpose_category: {df['purpose_category'].sort_values().value_counts()}")

Уникальные значения столбца purpose_category: недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_category, dtype: int64


По количеству детей можно создать 3 категории: "нет детей", "малодетная семья" (1 или 2 ребенка, в России многодетными считаются семьи, где 3 и более детей) и "многодетная семья".
Создадив функцию, которой будем передавать строку и по количеству детей определим категорию:

In [53]:
def children_category(row):
    """
    Разделяет по 3 категориям в зависимости от количества детей
    """
    children = row['children'] 
    
    if children == 0:
        return 'нет детей'
    
    elif children < 3:
        return 'малодетная семья'
    
    else: 
        return 'многодетная семья'

In [54]:
df['children_category'] = df.apply(children_category, axis=1)

Проверим, что все строки получили свою категорию и уникальные значения нового столбца:

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 14 columns):
children             21454 non-null int64
days_employed        21454 non-null int64
dob_years            21454 non-null int64
education            21454 non-null object
education_id         21454 non-null int64
family_status        21454 non-null object
family_status_id     21454 non-null int64
gender               21454 non-null object
income_type          21454 non-null object
debt                 21454 non-null int64
total_income         21454 non-null int64
purpose              21454 non-null object
purpose_category     21454 non-null object
children_category    21454 non-null object
dtypes: int64(7), object(7)
memory usage: 2.3+ MB


In [56]:
df['children_category'].value_counts()

нет детей            14091
малодетная семья      6983
многодетная семья      380
Name: children_category, dtype: int64

Для определения категорий по доходам, еще раз посмотрим на медиану по категориям занятости:

In [57]:
print('Медиана по доходам:', df.groupby('income_type')['total_income'].median())

Медиана по доходам: income_type
безработный        131339
в декрете           53829
госслужащий        150447
компаньон          172357
пенсионер          118514
предприниматель    499163
сотрудник          142594
студент             98201
Name: total_income, dtype: int64


In [58]:
df['total_income'].median()

142594.0

Примем за медиану 145.000, тогда можно создать 4 группы:
1. 0,75 от медианы, то есть 110.000
2. от 0,75 до 1,25 медианы, то есть от 110.000 до 180.000
3. от 1.25 медианы до 2 медиан, то есть от 180.000 до 290.000
4. выше 2 медиан, то есть от 290.000

(видимо, все живут в Москве, раз такая медиана)

Создадив функцию, которой будем передавать строку и по достатку определим категорию:

In [59]:
def income_category(row):
    """
    Разделяет по 4 категориям по уровню дохода
    """
    income = row['total_income'] 
    
    if income < 110000:
        return 'доход до 110 тысяч'
    
    elif income < 180000:
        return 'доход от 110 до 180 тысяч'
    
    elif income < 290000:
        return 'доход от 180 до 290 тысяч'
    
    else: 
        return 'доход от 290 тысяч'

In [60]:
df['income_category'] = df.apply(income_category, axis=1)

Проверим, что все строки получили свою категорию и уникальные значения нового столбца:

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 15 columns):
children             21454 non-null int64
days_employed        21454 non-null int64
dob_years            21454 non-null int64
education            21454 non-null object
education_id         21454 non-null int64
family_status        21454 non-null object
family_status_id     21454 non-null int64
gender               21454 non-null object
income_type          21454 non-null object
debt                 21454 non-null int64
total_income         21454 non-null int64
purpose              21454 non-null object
purpose_category     21454 non-null object
children_category    21454 non-null object
income_category      21454 non-null object
dtypes: int64(7), object(8)
memory usage: 2.5+ MB


In [62]:
df['income_category'].value_counts()

доход от 110 до 180 тысяч    9389
доход до 110 тысяч           5642
доход от 180 до 290 тысяч    4767
доход от 290 тысяч           1656
Name: income_category, dtype: int64

**Вывод**

Данные с категориями подготовлены к ответам на вопросы.


Теперь есть категории:
* по цели кредита
* по уровню дохода 
* по типу семьи (количество детей)


Были категории:
* по семейному положению
* по наличию задолженностей

## Шаг 3. Ответьте на вопросы
<a id='step_3'></a>

- **Есть ли зависимость между наличием детей и возвратом кредита в срок?**

Для ответа на этот вопрос необходимо создать таблицу по категориям количества детей, а также добавим столбцы с количеством возвратов кредита, невозвратом и рассчитаем соотношение возврата к общему числу:

In [63]:
#df_1 = df.pivot_table(index='children_category', values='debt', aggfunc=['count', 'sum']).reset_index()
#df_1['returned_debt'] = df['count']['debt'] - df['sum']['debt']

df_1 = df.groupby(['children_category']).agg({'debt':['count','sum']})
df_1['returned_debt'] = df_1['debt']['count'] - df_1['debt']['sum']
df_1['ratio_debt'] = df_1['returned_debt']/df_1['debt']['count']
display(df_1)

debt       returned_debt ratio_debt
                   count   sum                         
children_category                                      
малодетная семья    6983   647          6336   0.907346
многодетная семья    380    31           349   0.918421
нет детей          14091  1063         13028   0.924562

**Вывод**

Зависимость между наличием детей и возвратом кредита в срок действительно есть:

1. Семьи без детей возвращают 92,45% кредитов, это самое высокое значение
2. На втором месте многодетные семьи, они возвращают 91,84% кредитов
3. Семьи с 1 или 2 детьми возвращают только 90,73% кредитов.


Это можно объяснить так:
* Семьям без детей легче всего возвращать долг банку, так как их траты ограничены 2 людьми.
* Многодетные семьи тщательнее контролируют свои расходы, они имеют опыт в распределении доходов и знают, сколько нужно на каждого члена семьи.
* Семьи с 1 или 2 детьми пока еще не знают, сколько точно нужно бюджета на ребенка/детей и часто неправильно рассчитывают бюджет, отсюда появляются просрочки по кредиту.

- **Есть ли зависимость между семейным положением и возвратом кредита в срок?**

Для ответа на этот вопрос необходимо создать таблицу по семейному положению, а также добавим столбцы с количеством возвратов кредита, невозвратов и рассчитаем соотношение возврата к общему числу:

In [94]:
df_2 = df.groupby(['family_status']).agg({'debt':['count','sum']})
df_2['returned_debt'] = df_2['debt']['count'] - df_2['debt']['sum']
df_2['ratio_debt'] = df_2['returned_debt']/df_2['debt']['count']
display(df_2)

debt      returned_debt ratio_debt
                       count  sum                         
family_status                                             
в разводе               1195   85          1110   0.928870
вдовец / вдова           959   63           896   0.934307
гражданский брак        4151  388          3763   0.906529
женат / замужем        12339  931         11408   0.924548
не женат / не замужем   2810  274          2536   0.902491

**Вывод**

Зависимость между семейным положением и возвратом кредита в срок есть:
1. Самый высокий процент возврата показали "вдовец / вдова": 93,43%
2. Потом идет категория "в разводе": 92,89%
3. Чуть меньше предыдущей категории "женат / замужем", они возвращают 92,46% кредитов
4. Намного меньше процент возврата у людей в гражданском браке (чтобы это ни значило): 90,65%
5. И чуть меньше, чем в предыдщей категории у людей "не женат / не замужем": 90,25%


Можно объяснить такой результат так:
* Вдовцы/вдовы, а также "в разводе" - это люди, которые остались одни по разным причинам, но, находясь в официальных отношениях осознали свою ответственность за финансы (так как теперь нельзя положиться на супруга) и умеют распределять доходы и расходы.
* Женатые и замужние имеют поддержку в лице супруга, поэтому их процент возвратов кредита также высок.
* Одинокие люди и люди в гражданском браке, возможно, еще не осознали как им правильно распоряжаться финансами или нет помощи с долгами, поэтому их процент невозврата самый высокий.

- **Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

Для ответа на этот вопрос необходимо создать таблицу по категории доходов, а также добавим столбцы с количеством возвратов кредита, невозвратов и рассчитаем соотношение возврата к общему числу:

In [95]:
df_3 = df.groupby(['income_category']).agg({'debt':['count','sum']})
df_3['returned_debt'] = df_3['debt']['count'] - df_3['debt']['sum']
df_3['ratio_debt'] = df_3['returned_debt']/df_3['debt']['count']
display(df_3)

debt      returned_debt ratio_debt
                          count  sum                         
income_category                                              
доход до 110 тысяч         5642  455          5187   0.919355
доход от 110 до 180 тысяч  9389  808          8581   0.913942
доход от 180 до 290 тысяч  4767  358          4409   0.924900
доход от 290 тысяч         1656  120          1536   0.927536

**Вывод**

Зависимость между уровнем дохода и возвратом кредита в срок также имеется:
1. Всех больше кредиты возвращают те, у кого доход больше 290 тысяч - 92,75%
2. Далее идут те, чей дохо находится в диапазоне от 180 до 290 тысяч - 92,49%
3. Меньше возвращают кредитов люди с доходом до 110 тысяч - 91,94%
4. Самый большой долг у тех, чей достаток варьируется от 110 до 180 тысяч - 91,39%

Можно объяснить это так:
* Те, чей доход превышает 180 тысяч рублей в месяц понимают риски взятия денег в кредит и это не влияет на их уровень жизни, просрочки связаны скорее всего с наличием нескольких крупных кредитов
* Также те, чей доход меньше 110 тысяч имеют меньше пророчек, потому что планируют расходы, при взятии долга расчитывают, какую часть дохода им нужно будет возвращать, чем те, у кого доход находится в диапазоне от 110 до 180 тысяч, потому что они могут взять долг,не расчитав отношение долга к уровню дохода, пытаясь сохранить свой уровень жизни без долгов

- **Как разные цели кредита влияют на его возврат в срок?**

Для ответа на этот вопрос необходимо создать таблицу по категории целей, а также добавим столбцы с количеством возвратов кредита, невозвратом и рассчитаем соотношение возврата к общему числу:

In [96]:
df_4 = df.groupby(['purpose_category']).agg({'debt':['count','sum']})
df_4['returned_debt'] = df_4['debt']['count'] - df_4['debt']['sum']
df_4['ratio_debt'] = df_4['returned_debt']/df_4['debt']['count']
display(df_4)

debt      returned_debt ratio_debt
                  count  sum                         
purpose_category                                     
автомобиль         4306  403          3903   0.906410
недвижимость      10811  782         10029   0.927666
образование        4013  370          3643   0.907800
свадьба            2324  186          2138   0.919966

**Вывод**

Разные цели кредита влияют на его возврат в срок:
1. Самый возвращаемый кредит (после проведения лемматизации мы также увидели, что и самый популярный) это категория недвижимости - 92,77%
2. Далее идет свадьба - 92%
3. На третьем месте образование с 90,78%
4. Чуть меньше предыдущей категории автомобили - 90,64%

Можно объснить следующими причинами:
* Покупка недвижимости очень важный вопрос в жизни людей, чаще всего это ипотека на 25-30 лет, люди берут ее, понимая ответственноть и волнуясь за свое жилье, ведь при невыплате банк может выставить квартиру на торги, никто не хочет потерять (часто) свое единственное жилье.
* Хороший процент возврата кредита на свадьбу связан с тем, что обычно это не очень большая сумма, также многие молодожены, возможно, гасят кредит после проведения мероприятия и получения денежных сумм в качестве подарка от родных и друзей.
* Кредит на образование довольно тяжело выплатить, так как тяжело совмещать работу и учебу, поэтому много просрочек
* Кредит на автомобиль предполагает повышения ежемесячных расходов помимо выплаты по кредиту (нужно сразу офомрить ОСАГО, возможно купить резину), траты на бензин, ремонт (если поддержанный автомобиль), поэтому эта категория с самым большим количеством задолженностей.

## Шаг 4. Общий вывод
<a id='step_4'></a>

Исходя из проанализированных данных, можно сделать вывод об идеальном заемщике:
Это человек, не обремененный детьми, который потерял супруга, имеющий доход более 290 тысяч и который берет в кредит недвижимость.

И также можно сформулировать критерии для заемщика, который, возможно, будет иметь просрочки по выданному кредиту: 
Человек не замужем/не женат, с доходом от 110 до 180 тысяч, берущий в кредит автомобиль (и, возможно, с 1 или 2 детьми).

Общий вывод по категориям:
1. Наличие 1-2 детей является фактором, который может препятствовать регулярной выплате кредита, так как в первую очередь нужно обеспечить всем необходимым именно детей.
2. Люди, не состоящие в официальном браке или одинокие часто имеют просрочки из-за того, что у них либо меньше ответсвенность перед банком, либо нет опоры, кто бы мог им помочь.
3. Если доход выше медианного, то выплачивать кредит намного проще, чем тем, кто имеет медианный доход (правда, там и количество кредитов выше), но и те, чей доход ниже медианного имеет меньше просрочек.
4. Кредит на автомобиль самая рискованная категория (для банка), так как люди часто покупают его на "последние" деньги, не заботясь о сопутствующих расходах.